# Imports and Usefull Functions

In [ ]:
import numpy as np
import pyvista as pv
from PIL import Image
import tensorflow as tf
import keras.backend as K
from keras.models import Model
import matplotlib.pyplot as plt
from tensorflow.keras import layers

# Load feature and target data
def loadDataset(dataSetDir = './SecondDataSets/TrainingDataSets/gosplData.npz'):
    dataSet = np.load(dataSetDir)
    features = dataSet['features']
    targets = dataSet['targets']
    return features, targets

#Coordinate transformation from cartesian to polar
def cartesianToPolarCoords(XYZ, useLonLat=True):
    X, Y, Z = XYZ[:, 0], XYZ[:, 1], XYZ[:, 2]
    R = (X**2 + Y**2 + Z**2)**0.5
    theta = np.arctan2(Y, X)
    phi = np.arccos(Z / R)

    # Return results either in spherical polar or leave it in radians
    if useLonLat == True:
        theta, phi = np.degrees(theta), np.degrees(phi)
        lon, lat = theta - 180, 90 - phi
        lon[lon < -180] = lon[lon < -180] + 360
        return R, lon, lat
    else:
        return R, theta, phi

#Coordinate transformation from spherical polar to cartesian
def polarToCartesian(radius, theta, phi, useLonLat=True):
    if useLonLat == True:
        theta, phi = np.radians(theta+180.), np.radians(90. - phi)
    X = radius * np.cos(theta) * np.sin(phi)
    Y = radius * np.sin(theta) * np.sin(phi)
    Z = radius * np.cos(phi)

    # Return data either as a list of XYZ coordinates or as a single XYZ coordinate
    if (type(X) == np.ndarray):
        return np.stack((X, Y, Z), axis=1)
    else:
        return np.array([X, Y, Z])

# Visualizing the Training Data

Before we continue, we visualize the data to make sure that they are in the correct format and haven't been corrupted along the way. These functions will also be usefull to visualize the output of the tensorflow models later on.

In the code bellow, we provide a visualization of the data in the image format. This format is more representative of how tensorflow will actually see the data.

In [ ]:
# Display an image representation of a sample image
def visualizeAsImages(featureImage, targetImage):
    featureImage -= np.min(featureImage, axis=(0, 1))
    featureImage /= np.max(featureImage, axis=(0, 1))
    targetImage -= np.min(targetImage, axis=(0, 1))
    targetImage /= np.max(targetImage, axis=(0, 1))
    targetImage[:, :, 2] **= 0.125

    #Set up plotting figure
    fig, axis = plt.subplots(1, 3)
    fig.set_figheight(12)
    fig.set_figwidth(18)
    axis[0].imshow(featureImage[:, :, 0])
    axis[1].imshow(featureImage[:, :, 1])
    axis[2].imshow(targetImage)
    axis[0].set_title('Initial Elevations')
    axis[1].set_title('Tectonic Uplift')
    axis[2].set_title('Combined Output Data')
    axis[0].set_xticks([])
    axis[0].set_yticks([])
    axis[1].set_xticks([])
    axis[1].set_yticks([])
    axis[2].set_xticks([])
    axis[2].set_yticks([])
    plt.subplots_adjust(wspace=0.1, hspace=0.1)
    plt.show()

# Load dataset and visualize a sample from it
features, targets = loadDataset()
visualizeAsImages(features[0], targets[0])

Although the data used will be in an image format, it is meant to represent a spherical planet. The code bellow provides a spherical representation of what the data looks like, with an exagerated terrain based on the final elevations. This function will also be compatible with the outputs of the tensorflow models.

In the top left corner of the ITK plotter window, there is a dropdown meny represented by 3 lines. Within this dropdown menu, we can chose which parameter to color code the mesh with.

In [ ]:
# Create pyvista mesh object with exagerated terrains and data attached to it
def createExageratedMesh(featureSample, targetSample, amplificationFactor=90, earthRadius=6378137):
    
    # Reshape sample data and re-insert placeholder values (zeros) for north and south poles
    imageDims = np.array(features.shape[1:3])
    featureSample = featureSample.reshape(imageDims[0] * imageDims[1], 2)
    featureSample = np.insert(featureSample, 0, 0, axis=0)
    featureSample = np.insert(featureSample, 0, 0, axis=0)
    targetSample = targetSample.reshape(imageDims[0] * imageDims[1], 3)
    targetSample = np.insert(targetSample, 0, 0, axis=0)
    targetSample = np.insert(targetSample, 0, 0, axis=0)

    # Create a planet mesh with exagerated final elevations
    uvSphere = pv.Sphere(theta_resolution=imageDims[0], phi_resolution=imageDims[1]+2)
    r, lon, lat = cartesianToPolarCoords(uvSphere.points)
    exegeratedRadius = earthRadius + amplificationFactor * targetSample[:, 0]
    exageratedXYZ = polarToCartesian(exegeratedRadius, lon, lat)
    exageratedMesh = pv.PolyData(exageratedXYZ.T, uvSphere.faces)

    # Attach data to the exagerated mesh
    exageratedMesh['Initial Elevations'] = featureSample[:, 0]
    exageratedMesh['Tectonic Uplift'] = featureSample[:, 1]
    exageratedMesh['Final Elevations'] = targetSample[:, 0]
    exageratedMesh['Erosion Deposition'] = targetSample[:, 1]
    exageratedMesh['Flow Accumilation'] = targetSample[:, 2]**0.125
    return exageratedMesh

# Load data and create mesh
sampleNumber = 10
features, targets = loadDataset()
exageratedMesh = createExageratedMesh(features[sampleNumber], targets[sampleNumber])

# Plot the results
plotter = pv.PlotterITK()
plotter.add_mesh(exageratedMesh, scalars='Final Elevations')
plotter.show()

# Sigmoids and Logit Functions

Machine learning models generally don't perform well when the training data is within some random range, instead we want all data to be within a range of $[-1, 1]$, or sometimes within $[0, 1]$. We also don't want to use the standard normalization technique, because this would artificially introduce a lower and upper bound on our data.

We can use the sigmoid function to bring the data into the range of $[0, 1]$, and a logit function to bring it back to the original data domain. We refer to *gain* as the steepness of the sigmoid function.

In [ ]:
# Used to bring data to 0-1
def sigmoid(x, gain=1):
    return 1 / (1 + np.exp(- gain * x))

# Inverse function to bring data back to original domain
def logit(x, gain=1):
    return np.log(x / (1 - x)) / gain

# Create XY data to demonstrate functions with
x = np.arange(-10, 10, 0.01)
y = sigmoid(x, gain=1)
a = np.arange(0.01, 0.99, 0.01)
b = logit(a, gain=1)

# Plot functions
fig, axis = plt.subplots(1, 2)
fig.set_figheight(4)
fig.set_figwidth(16)
axis[0].plot(x, y)
axis[1].plot(a, b)
axis[0].set_title('Sigmoid')
axis[1].set_title('Logit')
plt.subplots_adjust(wspace=0.1, hspace=0.1)
plt.show()

When using the above functions, we want to choose an appropriate gain for each variable to avoid all values being too close to 0 or 1, but rather something inbetween. Otherwise the neural network may not *see* the relevant details of our data set.

We can use the figures bellow to figure out what range our data typically lies within, and decide what gain is suitable for each attribute.

In [ ]:
features, targets = loadDataset()
feats = features[0:5].reshape(5 * 512 * 256, 2)
targs = targets[0:5].reshape(5 * 512 * 256, 3)

fig, axis = plt.subplots(5, 1)
fig.set_figheight(12)
fig.set_figwidth(16)
axis[0].plot(feats[:, 0], linewidth=0.05)
axis[1].plot(feats[:, 1], linewidth=0.05)
axis[2].plot(targs[:, 0], linewidth=0.05)
axis[3].plot(targs[:, 1], linewidth=0.05)
axis[4].plot(targs[:, 2]**0.125, linewidth=0.05)
for i in range(5):
    axis[i].set_xticks([])

# The Unet Model

As discussed in our previous notebooks, the Unet model seems to be the most promising neural network architecture for our goal. We will explore the effectiveness of the Unet model when trained with a simple loss function and when trained in the context of a GAN.

The code bellow is taken and modified from [*nanoxas/sketch-to-terrain* Github](https://github.com/nanoxas/sketch-to-terrain/blob/master/model.py), where [Eric Guerin et. al.](https://hal.archives-ouvertes.fr/hal-01583706/file/tog.pdf) had a similar goal in mind.

In [ ]:
#Here we define the structure of the generator Unet model
def getUnet(imageDims):
    imageWidth, imageHeight = imageDims[1], imageDims[0]
    
    #Input layer
    inputs = layers.Input((imageHeight, imageWidth, 2))
    
    #A few convolutional layers with maxpooling
    #This is the encoder part of the model that interprets the input image
    conv1 = layers.Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = layers.Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = layers.MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = layers.Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = layers.Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = layers.MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = layers.Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = layers.Conv2D(256, 3, activation='relu', padding='same')(conv3)
    pool3 = layers.MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = layers.Conv2D(512, 3, activation='relu', padding='same')(pool3)
    conv4 = layers.Conv2D(512, 3, activation='relu', padding='same')(conv4)
    pool4 = layers.MaxPooling2D(pool_size=(2, 2))(conv4)
    conv5 = layers.Conv2D(1024, 3, activation='relu', padding='same')(pool4)
    conv5 = layers.Conv2D(1024, 3, activation='relu', padding='same')(conv5)
    
    #Their last convolution layer in their encoder seems to have skip layer with noise introduced to it
    #Noise is often used to avoid the overfiting of a machine learning model
    noise = layers.Input((K.int_shape(conv5)[1], K.int_shape(conv5)[2], K.int_shape(conv5)[3]))
    conv5 = layers.Concatenate()([conv5, noise])
    
    #From my understanding, upsampling is used to increase the weight of data in the minority class
    #Skip layer (connects conv4 layer directly to up6 layer), and more convolutional layers
    up6 = layers.Conv2D(512, 2, activation='relu', padding='same')(layers.UpSampling2D(size=(2, 2))(conv5))
    merge6 = layers.Concatenate()([conv4, up6])
    conv6 = layers.Conv2D(512, 3, activation='relu', padding='same')(merge6)
    conv6 = layers.Conv2D(512, 3, activation='relu', padding='same')(conv6)
    
    up7 = layers.Conv2D(256, 2, activation='relu', padding='same')(layers.UpSampling2D(size=(2, 2))(conv6))
    merge7 = layers.Concatenate()([conv3, up7])
    conv7 = layers.Conv2D(256, 3, activation='relu', padding='same')(merge7)
    conv7 = layers.Conv2D(256, 3, activation='relu', padding='same')(conv7)
    
    up8 = layers.Conv2D(128, 2, activation='relu', padding='same')(layers.UpSampling2D(size=(2, 2))(conv7))
    merge8 = layers.Concatenate()([conv2, up8])
    conv8 = layers.Conv2D(128, 3, activation='relu', padding='same')(merge8)
    conv8 = layers.Conv2D(128, 3, activation='relu', padding='same')(conv8)
    
    up9 = layers.Conv2D(64, 2, activation='relu', padding='same')(layers.UpSampling2D(size=(2, 2))(conv8))
    merge9 = layers.Concatenate()([conv1, up9])
    conv9 = layers.Conv2D(64, 3, activation='relu', padding='same')(merge9)
    conv9 = layers.Conv2D(64, 3, activation='relu', padding='same')(conv9)
    conv9 = layers.Conv2D(32, 3, activation='relu', padding='same')(conv9)
    conv10 = layers.Conv2D(3, 1, activation='tanh')(conv9)
    
    #Create and return the final model
    model = Model(inputs=[inputs, noise], outputs=conv10)
    return model


features, targets = loadDataset()
imageDims = features.shape[1:3]
generator = getUnet(imageDims)
generator.summary()

In [ ]:
batchSize = 2
featuresBatch = features[0:batchSize]
noise = np.random.normal(0, 1, (batchSize, 32, 16, 1024))
generatedImageBatch = generator([featuresBatch, noise], training=False).numpy()

#print(generatedImageBatch)
print(np.min(generatedImageBatch))
print(np.max(generatedImageBatch))

visualizeAsImages(featuresBatch[0], generatedImageBatch[0])

'''
exageratedMesh = createExageratedMesh(featuresBatch[0], generatedImageBatch[0])

# Plot the results
plotter = pv.PlotterITK()
plotter.add_mesh(exageratedMesh, scalars='Final Elevations')
plotter.show()
'''